In [1]:
import os
import sys

In [2]:
import pandas as pd

In [3]:
from importlib import reload
import time
from tqdm import tqdm

In [4]:
print(sys.version)

3.11.4 | packaged by conda-forge | (main, Jun 10 2023, 18:08:17) [GCC 12.2.0]


In [5]:
import enviserv.dictan
from enviserv.dictan import DictAnalyzer
import pandserv as pds # сервисные функции для пандас и не только

In [6]:
from sparkserv import SparkApp, Cols

In [7]:
import sparkserv

In [8]:
import pyspark

In [9]:
print(pyspark.__version__)

3.4.1


In [10]:
spark_app = SparkApp(my_logger_create_level = 'INFO')

INFO:spark_app:spark_master_ip: 172.18.0.2
INFO:spark_app:pyspark version: 3.4.1
INFO:spark_app:starting building spark app object: pyspark-taxi-forecasting
INFO:spark_app:Spark app object built as: <pyspark.sql.session.SparkSession object at 0x7f10dcfea090>
INFO:spark_app:Spark object can be accessed as the SparkApp_object.spark property


In [11]:
spark_master_ip = spark_app.get_spark_master_ip()
# print(spark_master_ip)

INFO:spark_app:spark_master_ip: 172.18.0.2


In [12]:
# spark = spark_app.build_spark_app(spark_master_ip=spark_master_ip)
spark = spark_app.spark

In [13]:
%%time
taxi = spark.read.load('/work/data/Taxi_Trips_-_2022.csv', 
                       format='csv', header='true', inferSchema='true')

CPU times: user 15.9 ms, sys: 1.8 ms, total: 17.7 ms
Wall time: 41.1 s


In [14]:
taxi_rdd = spark.sparkContext.textFile('/work/data/Taxi_Trips_-_2022.csv')

In [15]:
taxi_rdd.take(5)

['Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,Tolls,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid  Location',
 'bcfa19f2539021c054809d4c3993d226996ae095,368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334,01/01/2022 12:00:00 AM,01/01/2022 12:00:00 AM,152,0.1,,,,,3.75,0.00,0.00,0.00,3.75,Cash,Medallion Leasin,,,,,,',
 '2aba69ff015f9ea8e7bff43cab7eddb228f34a12,449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317,01/01/2022 12:00:00 AM,01/01/2022 12:30:00 AM,2360,17.44,,,,8,47.75,0.00,0.00,5.00,52.75,Cash,Flash Cab,,,,41.899602111,-87.633308037,POINT (-87.6333080367 41.899

In [16]:
%%time
# Вывести схему данных
taxi.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

In [17]:
pd.set_option('display.max_colwidth', None)  # Показать полное содержимое ячеек столбцов

In [18]:
# создадим объект для представления имен полей
ct = sparkserv.Cols(taxi)

ct.get_aliases()

{'trip_i': 'Trip ID',
 'taxi_i': 'Taxi ID',
 'trip_st': 'Trip Start Timestamp',
 'trip_et': 'Trip End Timestamp',
 'trip_s': 'Trip Seconds',
 'trip_m': 'Trip Miles',
 'pickup_ct': 'Pickup Census Tract',
 'dropoff_ct': 'Dropoff Census Tract',
 'pickup_ca': 'Pickup Community Area',
 'dropoff_ca': 'Dropoff Community Area',
 'fare': 'Fare',
 'tips': 'Tips',
 'tolls': 'Tolls',
 'extras': 'Extras',
 'trip_t': 'Trip Total',
 'payment_t': 'Payment Type',
 'company': 'Company',
 'pickup_cl': 'Pickup Centroid Latitude',
 'pickup_clon': 'Pickup Centroid Longitude',
 'pickup_cloc': 'Pickup Centroid Location',
 'dropoff_cl': 'Dropoff Centroid Latitude',
 'dropoff_clon': 'Dropoff Centroid Longitude',
 'dropoff_cloc': 'Dropoff Centroid  Location'}

In [19]:
table_descr = pd.read_csv('/work/data/table_descr.csv',sep=';',index_col='Column Name')

table_descr

,Description,Type,Описание,Тип
Column Name,,,,
Trip ID,A unique identifier for the trip.,Plain Text,Уникальный идентификатор поездки,Обычный текст
Taxi ID,A unique identifier for the taxi.,Plain Text,Уникальный идентификатор такси,Обычный текст
Trip Start Timestamp,When the trip started rounded to the nearest 15 minutes.,Date & Time,Время начала поездки округленное до ближайших 15 минут,Дата и время
Trip End Timestamp,When the trip ended rounded to the nearest 15 minutes.,Date & Time,Время окончания поездки округленное до ближайших 15 минут,Дата и время
Trip Seconds,Time of the trip in seconds.,Number,Продолжительность поездки в секундах,Число
Trip Miles,Distance of the trip in miles.,Number,Расстояние поездки в милях,Число
Pickup Census Tract,The Census Tract where the trip began. For privacy this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.,Plain Text,Код района по переписи населения где началась поездка. В целях конфиденциальности этот код не отображается для некоторых поездок. Это поле часто остается пустым для мест за пределами Чикаго.,Обычный текст
Dropoff Census Tract,The Census Tract where the trip ended. For privacy this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.,Plain Text,Код района по переписи населения где завершилась поездка. В целях конфиденциальности этот код не отображается для некоторых поездок. Это поле часто остается пустым для мест за пределами Чикаго.,Обычный текст
Pickup Community Area,The Community Area where the trip began. This column will be blank for locations outside Chicago.,Number,Номер коммунальной области где началась поездка. Это поле остается пустым для мест за пределами Чикаго.,Число


In [20]:
pd.reset_option('display.max_colwidth')  # Сброс максимальной ширины столбцов к значению по умолчанию

In [21]:
# %%time
# describe_result = taxi.describe().toPandas()

# time cost i5 (7g), 4 workers (x) 2gb RAM 
# CPU times: user 352 ms, sys: 32.2 ms, total: 384 ms
# Wall time: 2min 7s

In [22]:
# output_file_path = '/work/data/2022_describe.csv'
# describe_result.to_csv(output_file_path, index=False) #transpose().

In [23]:
dr = pd.read_csv('/work/data/2022_describe.csv',sep=',',index_col='summary')

In [24]:
drf = dr.copy()
pds.get_df_formated(drf, '`',2, 10)

In [25]:
drf.transpose()

summary,count,mean,stddev,min,max
Trip ID,6`382`425,nan,nan,000000bb18,ffffff1aae
Taxi ID,6`382`425,nan,nan,0041f8f0c9,fff84aa08a
Trip Start Timestamp,6`382`425,nan,nan,01/01/2022,12/31/2022
Trip End Timestamp,6`382`213,nan,nan,01/01/2022,12/31/2022
Trip Seconds,6`380`960,1`198.21,1`895.66,0,86`341
Trip Miles,6`382`369,6.19,8.00,0,2`967.54
Pickup Census Tract,2`623`831,17`031`468`160.38,368`945.90,17`031`010`100,17`031`980`100
Dropoff Census Tract,2`675`331,17`031`411`846.86,345`773.49,17`031`010`100,17`031`980`100
Pickup Community Area,5`868`572,32.35,25.20,1,77
Dropoff Community Area,5`748`741,25.84,20.93,1,77


In [26]:
val = int(dr.transpose()['count'].astype(float).max())

In [27]:
print(pds.gvf(val))

6'382'425


In [28]:
%%time
row_count = taxi.count()

# time cost i5 (7g), 4 workers (x) 2gb RAM
# CPU times: user 5.85 ms, sys: 871 µs, total: 6.72 ms
# Wall time: 11.8 s

CPU times: user 4.99 ms, sys: 594 µs, total: 5.58 ms
Wall time: 12.2 s


In [29]:
print(pds.gvf(row_count))

6'382'425


### Немножк пандас

In [30]:
# способ преобразования object столбца и
# фильтрации по стоблцу с использование to_numeric
drt = dr.transpose()
drtf = drt[pd.to_numeric(drt['count'], errors='coerce') > row_count*0.99][['count']]
# отображение отформатированных данных
pds.get_df_formated(drtf,'`',2, 10)
print('==================',pds.gvf(row_count))
drtf

================== 6'382'425


summary,count
Trip ID,6`382`425
Taxi ID,6`382`425
Trip Start Timestamp,6`382`425
Trip End Timestamp,6`382`213
Trip Seconds,6`380`960
Trip Miles,6`382`369
Fare,6`378`889
Tips,6`378`889
Tolls,6`378`889
Extras,6`378`889


In [31]:
# способ преобразования object столбца и
# фильтрации по стоблцу с использование astype
# 
# можно сразу применять тип и фильтровать, но значения в поле не поменяют тип
drtf1 = dr.transpose()[dr.transpose()['count'].astype(int) > -1][['count']]
# 
# если нужны вычисления с полем, то надо создать дф
drt1 = dr.transpose()
drt1['count'] = drt1['count'].astype(float)
drt1['not_null']=drt1['count']/row_count
# drt1[['count','not_null']]

In [32]:
drtf1 = drt1[drt1['count'] > -1][['count','not_null']]
# display(drtf1)
# отображение отформатированных данных
pds.get_df_formated(drtf1,'`',2, 10)
display(drtf1)
print('================================')
print('==total rows count==',pds.gvf(row_count))
print('================================')
print('=== columns count======',int(len(drtf1)))
print('================================')

summary,count,not_null
Trip ID,6`382`425,1
Taxi ID,6`382`425,1
Trip Start Timestamp,6`382`425,1
Trip End Timestamp,6`382`213,1.00
Trip Seconds,6`380`960,1.00
Trip Miles,6`382`369,1.00
Pickup Census Tract,2`623`831,0.41
Dropoff Census Tract,2`675`331,0.42
Pickup Community Area,5`868`572,0.92
Dropoff Community Area,5`748`741,0.90


==total rows count== 6'382'425
=== columns count====== 23


### Описание "Census Tract" и "Community Area"

Поля "Census Tract" и "Community Area" представляют собой два различных уровня административных единиц:

**Census Tract:**  
Это статистические единицы, используемые в переписи населения в США, проводимой каждые 10 лет.
Census Tract обычно представляют собой географические области, населенные от 1 200 до 8 000 человек.
Они предназначены для обеспечения стандартизированных данных для анализа населения, доходов, расового и этнического состава и других социально-экономических параметров.

Далее для краткости буду называть данную адм.единицу - **Квартал**

**Community Area:**  
Это административные районы в городе Чикаго, разработанные городским правительством.
Community Areas используются для организации и управления городскими службами и ресурсами.
Обычно Community Areas выступают в качестве единиц анализа для изучения социальных и экономических показателей, таких как уровень преступности, доступность образования и здравоохранения и др.

Далее для краткости буду называть данную адм.единицу - **Район**

Community Area представляют собой административные районы, охватывающие большие части города, в то время как Census Tract более мелкие географические области, которые могут быть распределены по всему Community Area. Таким образом, Community Area охватывает несколько Census Tract и представляет собой более крупную административную единицу.

В идеальной ситуации каждый Census Tract должен быть частью только одной Community Area, чтобы избежать перекрытий и дублирования данных. Однако в реальности могут быть исключения из-за изменений границ административных районов или других факторов.

Видим, что  
- 5 полей из 23 не содержат пропусков. 8 полей заполнены более чем на 99%.  
- `Pickup Census Tract`, `Dropoff Census Tract` заполнены менее, чем на 50%
- соответствующие поля с геоданными заполнены более, чем на 90%

Поскольку прогноз строится для квартала, следует связать поля геоданных и кварталов. Попробую заполнить пропуски. В случае невозможности заполненения пропусков для кварталов, в качестве ключа-аггрегатора буду использовать геоточки.

### Частоты "Census Tract"

In [33]:
# Функция для вычисления частот
def frq(df, column):
    # Вычисляем частоту для каждого значения
    freq_df = df.groupBy(column).count().orderBy("count", ascending=False)
    return freq_df

In [34]:
%%time
pct_frq = frq(taxi,ct.pickup_ct)
pct_frq.show()

+-------------------+-------+
|Pickup Census Tract|  count|
+-------------------+-------+
|               null|3758594|
|        17031980000| 519802|
|        17031320100| 304178|
|        17031839100| 284131|
|        17031281900| 188178|
|        17031081500| 174424|
|        17031081403| 112656|
|        17031330100| 108691|
|        17031081401| 104198|
|        17031980100|  85226|
|        17031081201|  84359|
|        17031081300|  76482|
|        17031320400|  71039|
|        17031081700|  63163|
|        17031320600|  61120|
|        17031280100|  55753|
|        17031841000|  41201|
|        17031081800|  39526|
|        17031081402|  35200|
|        17031081600|  30245|
+-------------------+-------+
only showing top 20 rows

CPU times: user 0 ns, sys: 15.4 ms, total: 15.4 ms
Wall time: 17.5 s


In [35]:
%%time
dct_frq = frq(taxi,ct.dropoff_ct)
dct_frq.show()

+--------------------+-------+
|Dropoff Census Tract|  count|
+--------------------+-------+
|                null|3707094|
|         17031839100| 328855|
|         17031320100| 277879|
|         17031980000| 243652|
|         17031081500| 160251|
|         17031330100| 139399|
|         17031081403| 133973|
|         17031281900| 129117|
|         17031081700| 114125|
|         17031081401| 100596|
|         17031081201|  87949|
|         17031833000|  83912|
|         17031320400|  81112|
|         17031081800|  76118|
|         17031081300|  75221|
|         17031280100|  67209|
|         17031320600|  59928|
|         17031980100|  48559|
|         17031841000|  42231|
|         17031081600|  40664|
+--------------------+-------+
only showing top 20 rows

CPU times: user 9.54 ms, sys: 1.14 ms, total: 10.7 ms
Wall time: 15.3 s


In [36]:
%%time
pca_frq = frq(taxi,ct.pickup_ca)
pca_frq.show()

+---------------------+-------+
|Pickup Community Area|  count|
+---------------------+-------+
|                    8|1482215|
|                   32|1031010|
|                   76| 989907|
|                   28| 531280|
|                 null| 513853|
|                   33| 229646|
|                    6| 229347|
|                   56| 183449|
|                    7| 150637|
|                    3| 100679|
|                   77|  76036|
|                   24|  75849|
|                    2|  47266|
|                   41|  44448|
|                   35|  35625|
|                    1|  35251|
|                   43|  33720|
|                   38|  33032|
|                   22|  31390|
|                   16|  30553|
+---------------------+-------+
only showing top 20 rows

CPU times: user 12.5 ms, sys: 255 µs, total: 12.7 ms
Wall time: 15.3 s


### Уникальные значения для "Census Tract"

In [37]:
def compare_unique_values(df, field1, field2):
    # Извлекаем уникальные значения для каждого поля
    unique_values_field1 = df.select(field1).distinct().collect()
    unique_values_field2 = df.select(field2).distinct().collect()
    
    # Преобразуем результаты в множества
    set_field1 = {row[field1] for row in unique_values_field1}
    set_field2 = {row[field2] for row in unique_values_field2}
    
    # Находим значения, которые есть в set_field1, но нет в set_field2
    only_in_field1 = {value for value in set_field1 if value not in set_field2}
    
    # Находим значения, которые есть в set_field2, но нет в set_field1
    only_in_field2 = {value for value in set_field2 if value not in set_field1}
    
    return only_in_field1, only_in_field2, set_field1, set_field2

In [38]:
%%time
only_pickup, only_dropoff, pickup, dropoff = \
compare_unique_values(taxi, ct.pickup_ct,ct.dropoff_ct)

CPU times: user 49.5 ms, sys: 10.1 ms, total: 59.6 ms
Wall time: 30.4 s


In [39]:
print('`Pickup Census Tract` distinct count:',len(pickup), ' | only in pickup set:',len(only_pickup))
print('`Dropoff Census Tract` distinct count:',len(dropoff), ' | only in dropoff set:',len(only_dropoff))

`Pickup Census Tract` distinct count: 666  | only in pickup set: 34
`Dropoff Census Tract` distinct count: 812  | only in dropoff set: 180


### Тренировка отбора тэйком. Отбор только значений элементов строк (из объектов Row)

In [40]:
%%time
taxi.take(1)

CPU times: user 0 ns, sys: 6.53 ms, total: 6.53 ms
Wall time: 187 ms


[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None)]

In [41]:
taxi.take(3)[:2]

[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12', Taxi ID='449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='

In [42]:
%%time
# Отбор значений для одного элемента объекта Row (одно поле)
[row[ct.trip_i] for row in taxi.take(3)[:2]]

CPU times: user 8.91 ms, sys: 0 ns, total: 8.91 ms
Wall time: 131 ms


['bcfa19f2539021c054809d4c3993d226996ae095',
 '2aba69ff015f9ea8e7bff43cab7eddb228f34a12']

In [43]:
# список словарей
[{field: row[field] for field in [ct.trip_i, ct.taxi_i]} for row in taxi.take(3)[:2]]

[{'Trip ID': 'bcfa19f2539021c054809d4c3993d226996ae095',
  'Taxi ID': '368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334'},
 {'Trip ID': '2aba69ff015f9ea8e7bff43cab7eddb228f34a12',
  'Taxi ID': '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317'}]

In [44]:
# список списков для нескольких полей, несколько проходов
[[row[ct.trip_i] for row in taxi.take(3)[:2]], [row[ct.taxi_i] for row in taxi.take(3)[:2]]]

[['bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'],
 ['368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317']]

In [45]:
# список кортежей, один проход
list(zip(*[[row[ct.trip_i], row[ct.taxi_i]] for row in taxi.take(3)[:2]]))

[('bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 ('368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317')]

In [46]:
# список списков - один проход
[list(t) for t in zip(*[[row[ct.trip_i], row[ct.taxi_i]] for row in taxi.take(3)[:2]])]

[['bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'],
 ['368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317']]

In [47]:
# список списков - произвольный список полей один проход.
fields = [ct.trip_i, ct.taxi_i]

[list(t) for t in zip(*[[row[field] for field in fields] for row in taxi.take(3)[:2]])]

[['bcfa19f2539021c054809d4c3993d226996ae095',
  '2aba69ff015f9ea8e7bff43cab7eddb228f34a12'],
 ['368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334',
  '449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317']]

### Использование zip(*...) и генератора списка

***Использование `zip(*...)` и генератора списка*** для преобразования кортежей в списки имеет минимальные накладные расходы. Эти операции довольно быстры, так как они выполняются в памяти и не требуют значительных вычислительных ресурсов.

Вот краткий обзор того, как работают эти преобразования:

**zip(*...)**: Это операция, которая транспонирует список списков. Она создает итератор, который группирует элементы из каждого из входных списков. Это делается за один проход по данным, что делает его очень эффективным.
Генератор списка: Преобразование каждого кортежа в список с помощью генератора списка `[list(t) for t in ...]` также выполняется за один проход по данным и выполняет простое преобразование типов. Это эффективно и не требует значительных вычислительных затрат.  
Для небольших наборов данных (несколько десятков или сотен элементов) накладные расходы будут практически незаметными. Для больших наборов данных, таких как миллионы элементов, накладные расходы могут стать более заметными, но по-прежнему будут минимальными по сравнению с другими более затратными операциями (например, ввод-вывод на диск или сложные вычисления).

Чтобы продемонстрировать минимальные накладные расходы, давайте рассмотрим пример с измерением времени:

In [48]:
# Пример данных
rows = [{ct.trip_i: f'trip_id_{i}', ct.taxi_i: f'taxi_id_{i}'} for i in range(1000000)]

# Измеряем время выполнения преобразования
start_time = time.time()
[list(t) for t in zip(*[[row[ct.trip_i], row[ct.taxi_i]] for row in rows])]
end_time = time.time()

print("Время выполнения: ", end_time - start_time)

Время выполнения:  3.1412734985351562


В большинстве случаев такие преобразования в Python очень эффективны, и вам не стоит беспокоиться о них, если нет необходимости оптимизировать производительность до предела.

### Используем селект для отбора значений строк
Можно отобрать только нужные поля переде тэйком

In [49]:
taxi.select(ct.trip_i).take(4)

[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095'),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef')]

In [50]:
taxi.select(ct.trip_i).take(4)[1:3]

[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace')]

In [51]:
# список списков - произвольный список полей один проход.
fields = [ct.trip_i, ct.taxi_i]

[list(t) for t in zip(*[[row[field] for field in fields] \
                        for row in taxi.select(ct.trip_i,ct.taxi_i).take(4)[1:3]])]

[['2aba69ff015f9ea8e7bff43cab7eddb228f34a12',
  '54d812a0b88f8f9707825261014b3563a0a60ace'],
 ['449fa4909552757130d09d98ebc7770e2dd94579036b0e1958f92577ffc9fd6deca97bc1f229d2700865ce12b54d454ba96f973077386210ccbe44bfb7aac317',
  'f98ae5e71fdda8806710af321dce58002146886c013f411be2cd336e0c224e78ed85626eeee462f0d1f5e6b29a030514b95f7b1b8ca59888a10b52dfe55ddf99']]

### Отбор строк через фильтрацию по номерам

In [52]:
from pyspark.sql.functions import monotonically_increasing_id

In [53]:
%%time
taxi.select(ct.trip_i).filter((monotonically_increasing_id() >= 1) \
                              & (monotonically_increasing_id() <= 4)).collect()

CPU times: user 9.72 ms, sys: 10.2 ms, total: 19.9 ms
Wall time: 14.1 s


[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef'),
 Row(Trip ID='f1a650ee419b4e52d766432e6f86eca3457bfb37'),
 Row(Trip ID='040caea96573c5743668b138011bc157c7825e86')]

In [54]:
%%time
taxi.select(ct.trip_i).rdd.zipWithIndex().filter(lambda x: 1 <= x[1] <= 4).map(lambda x: x[0]).collect()

CPU times: user 56.7 ms, sys: 1.24 ms, total: 57.9 ms
Wall time: 45.7 s


[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef'),
 Row(Trip ID='f1a650ee419b4e52d766432e6f86eca3457bfb37')]

In [55]:
%%time
taxi.select(ct.trip_i).rdd.take(4)

CPU times: user 6.75 ms, sys: 9.53 ms, total: 16.3 ms
Wall time: 164 ms


[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095'),
 Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef')]

**Причины неэффективности RDD Approach**  

**Монолитический подход DataFrame API:**  
`DataFrame API` в Apache Spark значительно оптимизирован за счет планировщика `Catalyst` и исполняющего механизма `Tungsten`. Эти оптимизации отсутствуют в `RDD API`, что делает операции на RDD в целом менее эффективными.  
`DataFrame API` оптимизирован с использованием `Catalyst optimizer`. Он понимает логику операций и может оптимизировать выполнение, используя проекцию, фильтрацию, и другие оптимизации.
В первом подходе, используя `monotonically_increasing_id, Spark` может оптимизировать фильтрацию на основе индексов и выполнять ее более эффективно.  
**Отсутствие оптимизации в RDD API:**  
`RDD API` не имеет уровня оптимизации, как `DataFrame API`. При использовании RDD, вы обязаны выполнять множество шагов вручную, что делает код менее эффективным.  
В данном случае, `zipWithIndex` создает пару `(элемент, индекс)`, которая потом фильтруется и мапится обратно. Это добавляет значительные накладные расходы на распределенные вычисления и коммуникацию между узлами кластера.  
**Сериализация и Десериализация:**  
Операции с RDD включают сериализацию и десериализацию данных между узлами, что может увеличить время выполнения, особенно на больших наборах данных.

**Как улучшить производительность RDD**  
Хотя `DataFrame API` чаще всего предпочтительнее для большинства операций из-за своих оптимизаций, бывают случаи, когда RDD необходимы. Вот несколько советов, чтобы улучшить производительность RDD:  
Используйте `Partitions Wisely`:  
Убедитесь, что количество партиций соответствует размерам данных и ресурсам кластера.  
`Persistence`:  
Используйте методы persist() или cache() для данных, которые будут многократно использоваться.  
`Avoid Shuffling`:  
Минимизируйте количество операций, требующих перемещения данных между узлами (например, groupByKey, reduceByKey).

**Добавление индексов:**  
`zipWithIndex()` добавляет индексы **к каждому элементу** в RDD. Эта операция может потребовать **полной загрузки** данных и выполнения дополнительного шага для распределения индексов.
**Фильтрация:**  
`filter(lambda x: 1 <= x[1] <= 4)` фильтрует элементы на основе их индексов. Эта операция применима **к каждому элементу** после добавления индексов.

Без `zipWithIndex()` не обойтись, если нужно отбирать элементы по их порядковому номеру, так как RDD по своей природе неупорядочен. Однако, если использовать take и затем отбирать нужные элементы, можно обойтись меньшими затратами.

In [56]:
%%time
taxi.select(ct.trip_i).rdd.take(4)[1:]

CPU times: user 13.5 ms, sys: 319 µs, total: 13.8 ms
Wall time: 177 ms


[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef')]

In [57]:
taxi.select(ct.trip_i).limit(5).collect()[1:5]

[Row(Trip ID='2aba69ff015f9ea8e7bff43cab7eddb228f34a12'),
 Row(Trip ID='54d812a0b88f8f9707825261014b3563a0a60ace'),
 Row(Trip ID='7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef'),
 Row(Trip ID='f1a650ee419b4e52d766432e6f86eca3457bfb37')]

### Сравнение скорости выполнения запросов через DF API и SQL

In [58]:
%%time
pck = taxi.select(ct.pickup_ct).distinct().collect()

CPU times: user 18.6 ms, sys: 0 ns, total: 18.6 ms
Wall time: 14.3 s


In [59]:
%%time
taxi.createOrReplaceTempView("taxis")
query = "SELECT DISTINCT `Pickup Census Tract` FROM taxis"
result = spark.sql(query)
pck2 = result.collect()

CPU times: user 16.1 ms, sys: 1.53 ms, total: 17.7 ms
Wall time: 14.4 s


In [60]:
# Количество испытаний
num_trials = 5

# Время выполнения для первого блока кода: сбор уникальных значений с помощью DataFrame API
pck_times = []
for _ in tqdm(range(num_trials)):
    start_time = time.time()
    pck = taxi.select('Pickup Census Tract').distinct().collect()
    end_time = time.time()
    pck_times.append(end_time - start_time)

# Вывод среднего времени выполнения для первого блока кода
print("Среднее время выполнения для первого блока кода: {:.2f} секунд".format(sum(pck_times) / num_trials))

100%|██████████| 5/5 [01:12<00:00, 14.49s/it]

Среднее время выполнения для первого блока кода: 14.49 секунд


In [61]:
# Время выполнения для второго блока кода: сбор уникальных значений с помощью SQL запроса
result_times = []
for _ in tqdm(range(num_trials)):
    start_time = time.time()
    taxi.createOrReplaceTempView("taxis")
    result = spark.sql("SELECT DISTINCT `Pickup Census Tract` FROM taxis")
    pck2 = result.collect()
    end_time = time.time()
    result_times.append(end_time - start_time)

# Вывод среднего времени выполнения для второго блока кода
print("Среднее время выполнения для второго блока кода: {:.2f} секунд".format(sum(result_times) / num_trials))


100%|██████████| 5/5 [01:09<00:00, 13.99s/it]

Среднее время выполнения для второго блока кода: 13.98 секунд


**DF API по скорости сравнимо с SQL**

### Ключи-агрегаторы

### В качестве ключа предполагается использовать кортеж-тройку (код квартала, широта, долгота)

Проверю взаимнооднозначность множеств кварталов и геоточек (для непустых зачений)

In [94]:
%%time
# список уникальных троек
f1 = ct.pickup_ct # 'Pickup Census Tract'
f2 = ct.pickup_cl # 'Pickup Centroid Latitude'
f3 = ct.pickup_clon # 'Pickup Centroid Longitude'
pckgeo = taxi.select(f1, f2, f3).distinct().collect()

CPU times: user 27.5 ms, sys: 749 µs, total: 28.2 ms
Wall time: 27.4 s


In [63]:
len(pckgeo)

743

In [64]:
pckgeo[:3]

[Row(Pickup Census Tract=17031250300, Pickup Centroid Latitude=41.912836784, Pickup Centroid Longitude=-87.760884371),
 Row(Pickup Census Tract=17031831600, Pickup Centroid Latitude=41.925588088, Pickup Centroid Longitude=-87.797600684),
 Row(Pickup Census Tract=17031351000, Pickup Centroid Latitude=41.834673598, Pickup Centroid Longitude=-87.610834737)]

In [65]:
len([row for row in pckgeo if row[f1] is None])

78

In [66]:
from collections import defaultdict

In [67]:
# Инициализация словаря для хранения значений
tract_to_coords = defaultdict(list)

# Заполнение словаря
for row in tqdm(pckgeo):
    tract = row[f1]
    coords = (row[f3], row[f2])
    tract_to_coords[tract].append(coords)

# Поиск районов с разными значениями широты и долготы
repeated_tracts = {tract: coords for tract, coords in tract_to_coords.items() if len(set(coords)) > 1}

100%|██████████| 743/743 [00:00<00:00, 74907.29it/s]


In [68]:
# Вывод результатов
for tract, coords in repeated_tracts.items():
    print(f"Район: {tract}, Количетво геоточек: {len(coords)}")
    print()

# Количество таких районов
print(f"Количество районов с разными значениями широты и долготы: {len(repeated_tracts)}")

Район: None, Количетво геоточек: 78

Количество районов с разными значениями широты и долготы: 1


In [69]:
for geop in repeated_tracts[None]:
    if geop == (None,None): continue
    # print(f"geop {geop}")
    for tract, geos in tract_to_coords.items():
        if tract is None: continue
        if geop in geos:
            print(f"geopoint: {geop} - {tract}")

geopoint: (-87.596475596, 41.728182061) - 17031470100


In [70]:
# tract_to_coords

In [71]:
%%time
geos_pck = taxi.select(f2, f3).distinct().orderBy(f2, f3).collect()

CPU times: user 27.4 ms, sys: 699 µs, total: 28.1 ms
Wall time: 15.6 s


In [72]:
len(geos_pck)

616

In [73]:
geos_pck[:5]

[Row(Pickup Centroid Latitude=None, Pickup Centroid Longitude=None),
 Row(Pickup Centroid Latitude=41.651921576, Pickup Centroid Longitude=-87.564929171),
 Row(Pickup Centroid Latitude=41.660136051, Pickup Centroid Longitude=-87.60284764),
 Row(Pickup Centroid Latitude=41.663670652, Pickup Centroid Longitude=-87.540935513),
 Row(Pickup Centroid Latitude=41.663759731, Pickup Centroid Longitude=-87.637304085)]

In [74]:
%%time
f2 = 'Pickup Centroid Latitude'
f3 = 'Pickup Centroid Longitude'
geos_pck_not_null = taxi.select(f2, f3).filter(taxi[f2].isNotNull() &
                taxi[f3].isNotNull()).distinct().orderBy(taxi[f2].asc(), taxi[f3].asc()).collect()

CPU times: user 23 ms, sys: 2.33 ms, total: 25.3 ms
Wall time: 15.9 s


In [75]:
geos_pck_not_null[:5]

[Row(Pickup Centroid Latitude=41.651921576, Pickup Centroid Longitude=-87.564929171),
 Row(Pickup Centroid Latitude=41.660136051, Pickup Centroid Longitude=-87.60284764),
 Row(Pickup Centroid Latitude=41.663670652, Pickup Centroid Longitude=-87.540935513),
 Row(Pickup Centroid Latitude=41.663759731, Pickup Centroid Longitude=-87.637304085),
 Row(Pickup Centroid Latitude=41.665167734, Pickup Centroid Longitude=-87.535284961)]

In [76]:
%%time
geos_pck_not_null = taxi.select(f2, f3).filter(taxi[f2].isNull() &
                taxi[f3].isNull()).collect()

CPU times: user 3.74 s, sys: 84.4 ms, total: 3.82 s
Wall time: 18.6 s


In [77]:
%%time
geos_pck_not_null = taxi.select(f2, f3).filter(taxi[f2].isNull() & taxi[f3].isNull())

CPU times: user 172 ms, sys: 28.7 ms, total: 201 ms
Wall time: 216 ms


In [78]:
%%time
print(pds.gvf(geos_pck_not_null.count()))

511'551
CPU times: user 9.09 ms, sys: 620 µs, total: 9.71 ms
Wall time: 14.4 s


In [79]:
%%time
taxi.select(f2, f3).filter(taxi[f2].isNull() &
                taxi[f3].isNotNull()).count()

CPU times: user 6.92 ms, sys: 9.91 ms, total: 16.8 ms
Wall time: 13.6 s


0

In [80]:
%%time
taxi.select(f2, f3).filter(taxi[f2].isNotNull() &
                taxi[f3].isNull()).count()

CPU times: user 6.7 ms, sys: 4.51 ms, total: 11.2 ms
Wall time: 14.3 s


0

In [81]:
taxi.select(ct.pickup_ca).distinct().count()

78

In [82]:
taxi.select(ct.dropoff_ca).distinct().count()

78

In [83]:
ct.testtt()

*******-------------*******


In [84]:
ct.get_aliases()

{'trip_i': 'Trip ID',
 'taxi_i': 'Taxi ID',
 'trip_st': 'Trip Start Timestamp',
 'trip_et': 'Trip End Timestamp',
 'trip_s': 'Trip Seconds',
 'trip_m': 'Trip Miles',
 'pickup_ct': 'Pickup Census Tract',
 'dropoff_ct': 'Dropoff Census Tract',
 'pickup_ca': 'Pickup Community Area',
 'dropoff_ca': 'Dropoff Community Area',
 'fare': 'Fare',
 'tips': 'Tips',
 'tolls': 'Tolls',
 'extras': 'Extras',
 'trip_t': 'Trip Total',
 'payment_t': 'Payment Type',
 'company': 'Company',
 'pickup_cl': 'Pickup Centroid Latitude',
 'pickup_clon': 'Pickup Centroid Longitude',
 'pickup_cloc': 'Pickup Centroid Location',
 'dropoff_cl': 'Dropoff Centroid Latitude',
 'dropoff_clon': 'Dropoff Centroid Longitude',
 'dropoff_cloc': 'Dropoff Centroid  Location'}

In [95]:
taxi.take(1)

[Row(Trip ID='bcfa19f2539021c054809d4c3993d226996ae095', Taxi ID='368ce5511598af2cc07efdb68067d381174fdac1d47a2837d021e647f4c8febb3d9bf330e155845b8217b9b451c29b1b9ecf5d55d4bfe6cfd5d79ffb49b1f334', Trip Start Timestamp='01/01/2022 12:00:00 AM', Trip End Timestamp='01/01/2022 12:00:00 AM', Trip Seconds=152, Trip Miles=0.1, Pickup Census Tract=None, Dropoff Census Tract=None, Pickup Community Area=None, Dropoff Community Area=None, Fare=3.75, Tips=0.0, Tolls=0.0, Extras=0.0, Trip Total=3.75, Payment Type='Cash', Company='Medallion Leasin', Pickup Centroid Latitude=None, Pickup Centroid Longitude=None, Pickup Centroid Location=None, Dropoff Centroid Latitude=None, Dropoff Centroid Longitude=None, Dropoff Centroid  Location=None)]

In [86]:
taxi.filter(taxi[ct.trip_m]>10).count()

1777033

In [87]:
long = taxi.filter((taxi[ct.trip_m]>9.9) & (taxi[ct.trip_m]<10.1) 
                   & taxi[ct.pickup_cl].isNotNull() & taxi[ct.dropoff_cl].isNotNull() 
                   & taxi[ct.pickup_clon].isNotNull() & taxi[ct.dropoff_clon].isNotNull()
                  )

In [88]:
long.count()

20855

In [89]:
long = long.withColumn('Distance',(
                                ((taxi[ct.pickup_cl]-taxi[ct.dropoff_cl])**2 +
                                (taxi[ct.pickup_clon]-taxi[ct.dropoff_clon])**2))**0.5)

In [90]:
%%time
long.select(ct.pickup_cl,ct.dropoff_cl,ct.pickup_clon,ct.dropoff_clon,'Distance').take(10)

CPU times: user 0 ns, sys: 13.5 ms, total: 13.5 ms
Wall time: 245 ms


[Row(Pickup Centroid Latitude=41.901206994, Dropoff Centroid Latitude=41.901206994, Pickup Centroid Longitude=-87.676355989, Dropoff Centroid Longitude=-87.676355989, Distance=0.0),
 Row(Pickup Centroid Latitude=41.874005383, Dropoff Centroid Latitude=41.9867118, Pickup Centroid Longitude=-87.66351755, Dropoff Centroid Longitude=-87.663416405, Distance=0.11270646238477244),
 Row(Pickup Centroid Latitude=41.899602111, Dropoff Centroid Latitude=41.839086906, Pickup Centroid Longitude=-87.633308037, Dropoff Centroid Longitude=-87.714003807, Distance=0.10086573913913337),
 Row(Pickup Centroid Latitude=41.857183858, Dropoff Centroid Latitude=41.9867118, Pickup Centroid Longitude=-87.620334624, Dropoff Centroid Longitude=-87.663416405, Distance=0.1365046798204703),
 Row(Pickup Centroid Latitude=41.874005383, Dropoff Centroid Latitude=41.857183858, Pickup Centroid Longitude=-87.66351755, Dropoff Centroid Longitude=-87.620334624, Distance=0.046343595040167315),
 Row(Pickup Centroid Latitude=41